In [1]:
import wandb
from wandb import Api
import pandas as pd
import os
import shutil

ENTITY = "pothole-detector"
PROJECT = "pothole-detector-NatureSR"

# === LOGIN ===
wandb.login()

# === SETUP ===
api = Api()

wandb: Currently logged in as: lorenzo-arcioni to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## Download dei dati degli sweeps

In [ ]:
# === CONFIGURAZIONE ===
sweeps_mapping = {
    #'9b6kmt2s':'prova'
    "lc4v3n51" : "YOLOv11s", # Sweep Bayesian
    "69xj2wxu" : "YOLOv11s", # Sweep with the initial run
    "cluaz2iu": "YOLOv11n",  # Sweep Bayesian
    "9b6kmt2s": "YOLOv11n",  # Sweep with the initial run 
}

# === ESTRAZIONE DEI DATI ===
all_runs_data = []

summary_useful = [
    'lr/pg0', 
    'lr/pg1', 
    'lr/pg2', 
    'metrics/mAP50(B)', 
    'metrics/mAP50-95(B)', 
    'metrics/precision(B)', 
    'metrics/recall(B)', 
    'model/GFLOPs', 
    'model/parameters', 
    'model/speed_PyTorch(ms)', 
    'train/box_loss', 
    'train/cls_loss', 
    'train/dfl_loss',
    'val/box_loss',
    'val/cls_loss', 
    'val/dfl_loss',
]

# === Creiamo il dizionario con tutte le cartelle di ogni sweep ===
OUTPUT_DIR = {sweep_id: f"results/{name}" for sweep_id, name in sweeps_mapping.items()}

# === Creiamo la cartella per le configs ===
CONFIG_DIR = "configs"
os.makedirs(CONFIG_DIR, exist_ok=True)

# Funzione per scaricare un artifact e copiare i file che rispettano la condizione
def process_artifact(artifact, run_name, run_id, dest_dir, file_check, suffix):
    """Scarica un artifact e copia i file che rispettano la condizione `file_check`."""
    try:
        artifact_dir = artifact.download()
        print(f"   ⬇️ Scaricato in: {artifact_dir}")

        found_files = []
        for root, _, files in os.walk(artifact_dir):
            for file in files:
                if file_check(file):  # condizione personalizzata
                    src = os.path.join(root, file)
                    artifact_name_clean = artifact.name.replace(":", "_").replace("/", "_")
                    out_name = f"{run_name}_{run_id}_{artifact_name_clean}{suffix}"
                    dst = os.path.join(dest_dir, out_name)
                    shutil.copy(src, dst)
                    print(f"   ✅ Salvato: {dst}")
                    found_files.append(dst)
        return found_files

    except Exception as e:
        print(f"   ❌ Errore scaricando artifact {artifact.name}: {e}")
        return []

# === CICLO SU TUTTI GLI SWEEP ===
for sweep_id in sweeps_mapping:
    out_dir = OUTPUT_DIR[sweep_id]
    train_dir, val_dir = [os.path.join(out_dir, d) for d in ("train", "val")]
    for d in (out_dir, train_dir, val_dir):
        os.makedirs(d, exist_ok=True)

    print(f"\n📦 Sweep {sweep_id} → output in '{out_dir}'")

    try:
        sweep = api.sweep(f"{ENTITY}/{PROJECT}/{sweep_id}")
        runs = sweep.runs
        print(f"🔍 Trovati {len(runs)} run")

        for run in runs:
            run_id, run_name = run.id, run.name or run.id
            print(f"🔄 Analizzo run: {run_name}")

            # Scarica config.yaml se esiste
            try:
                if any(f.name == "config.yaml" for f in run.files()):
                    cfg_file = run.file("config.yaml")
                    cfg_path = os.path.join(CONFIG_DIR, f"{run_name}_{run_id}_config.yaml")
                    cfg_file.download(CONFIG_DIR, replace=True)
                    downloaded = os.path.join(CONFIG_DIR, "config.yaml")
                    if os.path.exists(downloaded):
                        os.rename(downloaded, cfg_path)
                    print(f"   ⚙️ Config salvato: {cfg_path}")
                else:
                    print("   ⚠️ Nessun config.yaml trovato per questo run")
            except Exception as e:
                print(f"   ❌ Errore scaricando config.yaml: {e}")

            # Ciclo artifacts
            found_files = []
            artifacts = run.logged_artifacts()
            print(f"   📁 Trovati {len(artifacts)} artifacts")

            for artifact in artifacts:
                base_name = artifact.name.split(':')[0]

                if base_name.endswith('_log'):
                    print(f"   🔍 Artifact training: {artifact.name}")
                    found_files += process_artifact(
                        artifact, run_name, run_id, train_dir,
                        lambda f: f == "results.csv", suffix="_results.csv"
                    )

                elif base_name.endswith('_results'):
                    print(f"   🔍 Artifact validazione: {artifact.name}")
                    found_files += process_artifact(
                        artifact, run_name, run_id, val_dir,
                        lambda f: f.endswith("_results.pt"), suffix="_results.pt"
                    )

            if not found_files:
                print("   ⚠️ Nessun file di risultati trovato.")
            else:
                print(f"   🎉 Trovati {len(found_files)} file")

    except Exception as e:
        print(f"❌ Errore nello sweep {sweep_id}: {e}")

print("\n🏁 Download completato!")


📦 Sweep lc4v3n51 → output in 'results/YOLOv11s'
🔍 Trovati 59 run
🔄 Analizzo run: clean-sweep-59
   ⚙️ Config salvato: configs/clean-sweep-59_ej3ztnh0_config.yaml
   📁 Trovati 8 artifacts
   🔍 Artifact validazione: fold0_results:v121


wandb:   1 of 1 files downloaded.  


   ⬇️ Scaricato in: /home/lorenzo/Documenti/GitHub/Computer-Science-Sapienza/Computer Vision/project/artifacts/fold0_results:v121
   ✅ Salvato: results/YOLOv11s/val/clean-sweep-59_ej3ztnh0_fold0_results_v121_results.pt
   🔍 Artifact training: fold0_log:v284


wandb:   1 of 1 files downloaded.  


   ⬇️ Scaricato in: /home/lorenzo/Documenti/GitHub/Computer-Science-Sapienza/Computer Vision/project/artifacts/fold0_log:v284
   ✅ Salvato: results/YOLOv11s/train/clean-sweep-59_ej3ztnh0_fold0_log_v284_results.csv
   🔍 Artifact validazione: fold1_results:v121


...

   ⬇️ Scaricato in: /home/lorenzo/Documenti/GitHub/Computer-Science-Sapienza/Computer Vision/project/artifacts/fold2_log:v169
   ✅ Salvato: results/YOLOv11n/train/neat-sweep-1_ptyk0oqv_fold2_log_v169_results.csv
   🔍 Artifact validazione: fold3_results:v2


wandb:   1 of 1 files downloaded.  


   ⬇️ Scaricato in: /home/lorenzo/Documenti/GitHub/Computer-Science-Sapienza/Computer Vision/project/artifacts/fold3_results:v2
   ✅ Salvato: results/YOLOv11n/val/neat-sweep-1_ptyk0oqv_fold3_results_v2_results.pt
   🔍 Artifact training: fold3_log:v169


wandb:   1 of 1 files downloaded.  


   ⬇️ Scaricato in: /home/lorenzo/Documenti/GitHub/Computer-Science-Sapienza/Computer Vision/project/artifacts/fold3_log:v169
   ✅ Salvato: results/YOLOv11n/train/neat-sweep-1_ptyk0oqv_fold3_log_v169_results.csv
   🎉 Trovati 8 file

🏁 Download completato!


## Download dei risultati di singoli runs

In [2]:
# === CONFIGURAZIONE ===
runs_mapping = {
    "q7f86le3": "MobileNetv3-SSD320",
    # "def456zw": "YOLOv11n_optimized",
    # "ghi789uv": "YOLOv11s_experiment1",
    # Aggiungi altri run qui
}

# === CARTELLE OUTPUT ===
OUTPUT_DIR = {run_id: f"results/{name}" for run_id, name in runs_mapping.items()}

# === CARTELLA CONFIG ===
CONFIG_DIR = "configs"
os.makedirs(CONFIG_DIR, exist_ok=True)

# Funzione per scaricare un artifact e copiare i file che rispettano la condizione
def process_artifact(artifact, run_name, run_id, dest_dir, file_check, suffix):
    """Scarica un artifact e copia i file che rispettano la condizione `file_check`."""
    try:
        artifact_dir = artifact.download()
        print(f"   ⬇️ Scaricato in: {artifact_dir}")

        found_files = []
        for root, _, files in os.walk(artifact_dir):
            for file in files:
                if file_check(file):  # condizione personalizzata
                    src = os.path.join(root, file)
                    artifact_name_clean = artifact.name.replace(":", "_").replace("/", "_")
                    out_name = f"{run_name}_{run_id}_{artifact_name_clean}{suffix}"
                    dst = os.path.join(dest_dir, out_name)
                    shutil.copy(src, dst)
                    print(f"   ✅ Salvato: {dst}")
                    found_files.append(dst)
        return found_files

    except Exception as e:
        print(f"   ❌ Errore scaricando artifact {artifact.name}: {e}")
        return []

# === CICLO SU TUTTI I RUN ===
for run_id, run_name in runs_mapping.items():
    out_dir = OUTPUT_DIR[run_id]
    train_dir, val_dir = [os.path.join(out_dir, d) for d in ("train", "val")]
    for d in (out_dir, train_dir, val_dir):
        os.makedirs(d, exist_ok=True)

    print(f"\n📦 Run {run_id} ({run_name}) → output in '{out_dir}'")

    try:
        # Recupera il run specifico
        run = api.run(f"{ENTITY}/{PROJECT}/{run_id}")
        print(f"🔍 Analizzando run: {run.name}")

        # Scarica config.yaml se esiste
        try:
            if any(f.name == "config.yaml" for f in run.files()):
                cfg_file = run.file("config.yaml")
                cfg_path = os.path.join(CONFIG_DIR, f"{run_name}_{run_id}_config.yaml")
                cfg_file.download(CONFIG_DIR, replace=True)
                downloaded = os.path.join(CONFIG_DIR, "config.yaml")
                if os.path.exists(downloaded):
                    os.rename(downloaded, cfg_path)
                print(f"   ⚙️ Config salvato: {cfg_path}")
            else:
                print("   ⚠️ Nessun config.yaml trovato per questo run")
        except Exception as e:
            print(f"   ❌ Errore scaricando config.yaml: {e}")

        # Ciclo artifacts
        found_files = []
        artifacts = run.logged_artifacts()
        print(f"   📁 Trovati {len(artifacts)} artifacts")

        for artifact in artifacts:
            base_name = artifact.name.split(':')[0]

            if base_name.endswith('_log'):
                print(f"   🔍 Artifact training: {artifact.name}")
                found_files += process_artifact(
                    artifact, run_name, run_id, train_dir,
                    lambda f: f == "results.csv", suffix="_results.csv"
                )

            elif base_name.endswith('_results'):
                print(f"   🔍 Artifact validazione: {artifact.name}")
                found_files += process_artifact(
                    artifact, run_name, run_id, val_dir,
                    lambda f: f.endswith("_results.pt"), suffix="_results.pt"
                )

        if not found_files:
            print("   ⚠️ Nessun file di risultati trovato.")
        else:
            print(f"   🎉 Trovati {len(found_files)} file")

    except Exception as e:
        print(f"❌ Errore nel run {run_id}: {e}")

print("\n🏁 Download completato!")


📦 Run q7f86le3 (MobileNetv3-SSD320) → output in 'results/MobileNetv3-SSD320'
🔍 Analizzando run: Mobilenetv3-ssd320_cv_4fold
   ⚙️ Config salvato: configs/MobileNetv3-SSD320_q7f86le3_config.yaml
   📁 Trovati 6 artifacts
   🔍 Artifact validazione: fold0_results:v127


wandb:   3 of 3 files downloaded.  


   ⬇️ Scaricato in: /home/lorenzo/Documenti/GitHub/Computer-Science-Sapienza/Computer Vision/project/artifacts/fold0_results:v127
   ✅ Salvato: results/MobileNetv3-SSD320/val/MobileNetv3-SSD320_q7f86le3_fold0_results_v127_results.pt
   🔍 Artifact validazione: fold1_results:v127


wandb:   3 of 3 files downloaded.  


   ⬇️ Scaricato in: /home/lorenzo/Documenti/GitHub/Computer-Science-Sapienza/Computer Vision/project/artifacts/fold1_results:v127
   ✅ Salvato: results/MobileNetv3-SSD320/val/MobileNetv3-SSD320_q7f86le3_fold1_results_v127_results.pt
   🔍 Artifact validazione: fold2_results:v130


wandb:   3 of 3 files downloaded.  


   ⬇️ Scaricato in: /home/lorenzo/Documenti/GitHub/Computer-Science-Sapienza/Computer Vision/project/artifacts/fold2_results:v130
   ✅ Salvato: results/MobileNetv3-SSD320/val/MobileNetv3-SSD320_q7f86le3_fold2_results_v130_results.pt
   🔍 Artifact validazione: fold3_results:v130


wandb:   3 of 3 files downloaded.  


   ⬇️ Scaricato in: /home/lorenzo/Documenti/GitHub/Computer-Science-Sapienza/Computer Vision/project/artifacts/fold3_results:v130
   ✅ Salvato: results/MobileNetv3-SSD320/val/MobileNetv3-SSD320_q7f86le3_fold3_results_v130_results.pt
   🎉 Trovati 4 file

🏁 Download completato!
